In [1]:
%%capture
!pip install -q langchain==0.3.10
!pip install -q langchain-openai==0.2.12
!pip install -q langchain-community==0.3.11
!pip -q install openai==1.55.3
!pip -q install faiss-cpu
!pip -q install tiktoken

In [2]:
import os
import getpass
from typing import Dict, Any

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# 🎯 **Zero-Shot Prompting with Large Language Models:**

- 🤖 **First Try, No Examples Needed**: Ask the AI to do something new without showing it how first.

- 🧠**Data-Driven Responses**: Thanks to its huge knowledge base, the AI can handle tasks straight away.

- ✅ **Example**: Just say, "Classify this text: neutral, negative, or positive?" and the AI gets to work.

- 🎯➡️🔍 **Plan B**: If the first shot doesn't hit the mark, try adding examples to guide the AI.

Zero-shot prompting is like asking a chef to whip up a dish they've never made before—they might just surprise you with what they can do without a recipe!


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

In [6]:
class ZeroShotChain:
    """
    A class to facilitate zero-shot tasks

    Attributes:
        llm: An instance of an LLM.
    """

    def __init__(self, llm: Any) -> None:
        """
        Initializes the ZeroShotChain with the provided LLM instance.

        Args:
            llm: An instance of an LLM.
        """
        self.llm = llm

    def create_prompt(self, template: str) -> PromptTemplate:
        """
        Creates a prompt from the provided template.

        Args:
            template: A string template for creating the prompt.

        Returns:
            An instance of the created prompt.
        """
        prompt = PromptTemplate.from_template(template)
        return prompt

    def create_chain(self, llm, prompt: str):
        """
        Creates an LLMChain using the provided prompt.

        Args:
            prompt: An instance of the created prompt.

        Returns:
            An instance of the LLMChain.
        """
        output_parser = StrOutputParser()
        chain = prompt | llm | output_parser
        return chain

    def invoke(self, template: str, user_input: Dict[str, str]) -> str:
        """
        Runs the zero-shot task using the provided template and user input.

        Args:
            template: A string template for creating the prompt.
            user_input: A dictionary containing the user input for the task.

        Returns:
            The result of the zero-shot task.
        """
        prompt = self.create_prompt(template)
        chain = self.create_chain(self.llm, prompt)
        return chain.invoke(user_input)

In [9]:
# Initialize the llm
llm_instance = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Create an instance of the ZeroShotChain class with the LLM instance.
chain = ZeroShotChain(llm_instance)

In [10]:
chain

In [11]:
chain.create_chain

<bound method ZeroShotChain.create_chain of <__main__.ZeroShotChain object at 0x000001D04D83BC50>>

In [13]:
# Define the template and user input for the sarcasm classification task.
sarcasm_template = """Classify the user statement, delimited by < >, as sarcastic or not sarcastic.
User statement: <{statement}>
"""
sarcasm_input = {"statement": "Oh, yippe! Another flat tire."}

# Run the sarcasm classification task.
result = chain.invoke(sarcasm_template, sarcasm_input)
print(result)

Sarcastic


In [14]:
# Define the template and user input for the appropriateness classification task.
appropriateness_template = """Classify the user statement, delimited by < >, as appropriate or inappropriate.
User statement: <{statement}>
"""
appropriateness_input = {"statement": "That was a very thoughtful comment."}

result = chain.invoke(appropriateness_template, appropriateness_input)
print(result)

Appropriate


In [15]:
sentiment_template = """Classify the user statement, delimited by < >, as positive or negative.
User statement: <{statement}>
"""
sentiment_input = {"statement": "I had an amazing day today!"}


result = chain.invoke(sentiment_template, sentiment_input)
print(result)

Positive


## 🔍 **Named Entity Recognition (NER):**


- **NER**: Identifies names, places, dates, etc., in text. 🕵️‍♂️
- **Use**: Quickly find key info. 🔍

NER acts like a scanner for important text details.


In [16]:
# Define the template and user input for the named entity recognition task.
entities_template = """Identify and categorize the named entities in the text delimited by <>.
Text: <{statement}>
"""
entities_input = {"statement": "Barack Obama was the 44th president of the United States. He was born in Honolulu, Hawaii, on August 4, 1961. Before his presidency, he attended Harvard Law School."}

# Run the named entity recognition task.
result = chain.invoke(entities_template, entities_input)

print(result)

Named entities:
- Barack Obama (Person)
- United States (Location)
- Honolulu, Hawaii (Location)
- August 4, 1961 (Date)
- Harvard Law School (Organization)


In [11]:
# Define the template and user input for resume generation.
resume_template = """Write a cover letter, in the most pompous way possible, /
that will guarantee the following person does not get hired:

Name: {name}
Contact Information: {contact_info}
Education: {education}
Work Experience: {work_experience}
Skills: {skills}
"""

resume_input = {
    "name": "Harpreet Sahota",
    "contact_info": "Email: harpreetsahota@example.com\nPhone: (555) 420-6969",
    "education": "Bachelor of Science in Computer Science, XYZ University, 2015-2019",
    "work_experience": "Software Engineer, ABC Tech, 2019-present\n- Developed web applications using Python and Django\n- Collaborated with cross-functional teams on project delivery",
    "skills": "Programming: Python, Java, C++\nWeb Development: HTML, CSS, JavaScript\nDatabase: SQL, MongoDB"
}


result = chain.invoke(resume_template, resume_input)

print(result)

[Your Name] [Your Address] [City, State, Zip Code] [Your Email Address] [Today’s Date]

[Recipient's Name] [Recipient's Title] [Company Name] [Company Address] [City, State, Zip Code]

Dear [Recipient's Name],

I am writing to express my interest in the [Job Title] position at [Company Name]. With my unparalleled skills and expertise in the field of computer science, I am confident that I am the perfect candidate for this role.

As a proud graduate of XYZ University with a Bachelor of Science in Computer Science, I have honed my abilities in programming and web development. My tenure as a Software Engineer at ABC Tech has further refined my talents, allowing me to develop cutting-edge web applications using Python and Django, and collaborate seamlessly with cross-functional teams for project delivery.

My extensive skill set in programming languages such as Python, Java, and C++, as well as my mastery of web development languages like HTML, CSS, and JavaScript, make me an invaluable as



# **Zero-Shot Prompting Limits:**

- 🎚️ **Less Control**: Can't fine-tune AI outputs.
- 🧩 **Complexity Struggle**: May falter on intricate tasks.
- 🎓 **Domain Hurdles**: Specialized topics can trip it up.
- 📏 **Length Matters**: Short texts work best.

Zero-shot prompting can be hit or miss—great for quick tasks, less so for nuanced or lengthy content.
